In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log, sqrt
%matplotlib inline
from sklearn import linear_model # using scikit-learn
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int,
'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float,
'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float,
'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str,
'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

all_features = ['bedrooms', 'bedrooms_square', 'bathrooms',
'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt',
'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade',
'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']

sales = pd.read_csv('kc_house_data.csv',dtype=dtype_dict)
X_train = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
X_test = pd.read_csv('kc_house_test_data.csv',dtype=dtype_dict)

In [2]:
def get_numpy_data(df, features, output):
    df['constant'] = 1 # add a constant column to an SFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    features_matrix = df[features].as_matrix()
    output_array = df[output].as_matrix()
    return(features_matrix, output_array)


In [3]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return predictions

In [4]:
def feature_derivative(errors, feature):
    derivative = 2* np.dot(errors, feature)
    return derivative

In [5]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalize_features = feature_matrix / norms
    return normalize_features, norms

# Effect of L1 penalty
## 9. Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.

- First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire ‘sales’ dataset for now.
- Normalize columns of the feature matrix. Save the norms of original features as ‘norms’.
- Set initial weights to [1,4,1].
- Make predictions with feature matrix and initial weights.
- Compute values of ro[i], where

ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]

In [6]:
ro =[]
simple_features = ['sqft_living','bedrooms']
feature_matrix, output = get_numpy_data(sales,simple_features,'price')
norm_feature_matrix, norms = normalize_features(feature_matrix)
initial_weights = [1. ,4. ,1.]
predictions = predict_output(norm_feature_matrix,initial_weights)
for i in range(norm_feature_matrix.shape[1]):
    ro.append(sum(norm_feature_matrix[:,i] * (output - predictions) +
                     norm_feature_matrix[:,i] * initial_weights[i]))
ro

[79400446.028127789, 87940004.766660035, 80966839.384621024]

### 10. Quiz Question: Recall that, whenever $\rho[i]$ falls between $\frac{-l1\_penalty}{2}$ and $\frac{l1\_penalty}{2}$, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?

<font color=red>For w[1] to be zero, we need $\rho[1]$ in [$-\lambda/2$, $\lambda/2$]
<br>
We have $-\frac{\lambda}{2} \leq \rho[1] \leq \frac{\lambda}{2}$
<br>
This translates to $\lambda \geq -2\rho[1]$ and $\lambda \geq 2\rho[1]$
<br>
For both conditions to be satisfied, $\lambda \geq 2\rho[1] = 1.75\times10^8$
<br>
Similarly for w[2]. $\lambda \geq 2\rho[2] = 1.62\times10^8$
<br>
So, $w[i] = 0$ if $\lambda \geq 2 \times|\rho[i]|$</font>



In [27]:
def in_l1range(value, penalty):
    return ( (value >= -penalty/2.) and (value <= penalty/2.) )

for l1_penalty in [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]:
    print( in_l1range(ro[1], l1_penalty), in_l1range(ro[2], l1_penalty))

False False
False True
False True
True True
True True


### 11. Quiz Question: What range of values of l1_penalty would set both w[1] and w[2] to zero, if we were to take a step in that coordinate?

In [29]:
def in_l1range(value, penalty):
    return ( (value >= -penalty/2.) and (value >= penalty/2.) )

for l1_penalty in [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]:
    print( in_l1range(ro[1], l1_penalty), in_l1range(ro[1], l1_penalty))

True True
True True
True True
False False
False False


In [39]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = sum((feature_matrix[:,i] * ((output - prediction) + feature_matrix[:,i]* weights[i])))
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [40]:
# should print 0.425558846691

import math
lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.42555884669102512

## Cyclical coordinate descent
### 13. Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:

- As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
- After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to the previous step.
- Return weights

The function should accept the following parameters:

- Feature matrix
- Output array
- Initial weights
- L1 penalty
- Tolerance

In [45]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    D = feature_matrix.shape[1]
    weights = np.array(initial_weights)
    change = np.array(initial_weights) * 0.0
    converged = False
    
    while not converged:
        for i in range(D):
            new_weight = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            change[i] = np.abs(new_weight - weights[i])
            weights[i] = new_weight
        max_change = max(change)
        if max_change < tolerance:
            converged = True
    return weights


In [46]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

### 15. Quiz Question: What is the RSS of the learned model on the normalized dataset?

### 16. Quiz Question: Which features had weight zero at convergence? 
<font color=red>feature 2: bedrooms</font>

In [47]:
simple_features_matrix, output = get_numpy_data(sales, simple_features, my_output)
norm_simple_features_matrix, simple_norms = normalize_features(simple_features_matrix)

In [49]:
weights = lasso_cyclical_coordinate_descent(norm_simple_features_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
weights

array([ 21624997.95951872,  63157247.20788978,         0.        ])

In [51]:
# RSS
RSS = np.dot(output - predict_output(norm_simple_features_matrix,weights),
             output - predict_output(norm_simple_features_matrix,weights))
RSS


1630492476715384.8

# Evaluating LASSO fit with more features


In [77]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']
feature_list=['w0']+all_features
feature_list

['w0',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

In [53]:
all_features_matrix, output = get_numpy_data(X_train, all_features, my_output)
norm_all_features_matrix, all_norms = normalize_features(all_features_matrix)

### 19. First, learn the weights with l1_penalty=1e7, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights’ weights1e7’, you will need them later.

In [78]:
l1_penalty = 1e7
initial_weights = np.zeros(len(feature_list))
tolerance = 1

weights1e7 = lasso_cyclical_coordinate_descent(norm_all_features_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

#### 20. Quiz Question: What features had non-zero weight in this case?



In [79]:
def non_zero_features(weights):
    answer =[]
    for i,n in enumerate(weights):
        if n != 0:
            answer.append(feature_list[i])  
    return answer
non_zero_features(weights1e7)

['w0', 'sqft_living', 'waterfront', 'view']

### 21. Next, learn the weights with l1_penalty=1e8, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights ‘weights1e8’, you will need them later.

#### 22. Quiz Question: What features had non-zero weight in this case?

In [81]:
l1_penalty = 1e8
initial_weights = np.zeros(len(feature_list))
tolerance = 1
weights1e8 = lasso_cyclical_coordinate_descent(norm_all_features_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
non_zero_features(weights1e8)

['w0']

### 23. Finally, learn the weights with l1_penalty=1e4, on the training data. Initialize weights to all zeros, and set the tolerance=5e5. Call resulting weights ‘weights1e4’, you will need them later. (This case will take quite a bit longer to converge than the others above.)

#### 24. Quiz Question: What features had non-zero weight in this case?

In [82]:
l1_penalty = 1e4
initial_weights = np.zeros(len(feature_list))
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(norm_all_features_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
non_zero_features(weights1e4)

['w0',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

# Rescaling learned weights


### 25. Recall that we normalized our feature matrix, before learning the weights. To use these weights on a test set, we must normalize the test data in the same way. Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data:

In this case, we must scale the resulting weights so that we can make predictions with original features:

- Store the norms of the original features to a vector called ‘norms’:
        features, norms = normalize_features(features)
- Run Lasso on the normalized features and obtain a ‘weights’ vector
- Compute the weights for the original features by performing element-wise division, i.e.
        weights_normalized = weights / norms

In [83]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(X_train, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
normalized_weights1e4 = weights1e4 / norms
print(normalized_weights1e7[3])

161.317457646


To check your results, if you call normalized_weights1e7 the normalized version of weights1e7, then:

print normalized_weights1e7[3]
should return 161.31745624837794.

# Evaluating each of the learned models on the test data¶


In [84]:
(test_feature_matrix, test_output) = get_numpy_data(X_test, all_features, 'price')


In [85]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e7)
RSS1e7 = np.dot(test_output-prediction, test_output-prediction)
prediction =  predict_output(test_feature_matrix, normalized_weights1e8)
RSS1e8 = np.dot(test_output-prediction, test_output-prediction)
prediction =  predict_output(test_feature_matrix, normalized_weights1e4)
RSS1e4 = np.dot(test_output-prediction, test_output-prediction)
print('Weight 1e7 RSS: ', RSS1e7)
print('Weight 1e8 RSS: ', RSS1e8)
print('Weight 1e4 RSS: ', RSS1e4)

Weight 1e7 RSS:  2.7596207592e+14
Weight 1e8 RSS:  5.37166151497e+14
Weight 1e4 RSS:  2.28459958971e+14


#### 28. Quiz Question: Which model performed best on the test data?

   <font color=red>1e4</font>